In [74]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from src.GGUFBot import GGUFBot
from src.HFDataset import HFDataset
from src.SimpleQuestionGenerator import SimpleQuestionGenerator
from src.AnswerGenerator import AnswerGenerator
import pandas as pd
import random


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:

#df=pd.read_csv('data/original_q.csv') # indexはなし
#original_question_list=df["question"].values.tolist()
#original_question_list


ds_name="hatakeyama-llm-team/WikiBookJa"
ds_name="allenai/peS2o"
ds_name="kanhatakeyama/TanukiCorpus"
ds_name="hatakeyama-llm-team/WikiBookJa"
ds=HFDataset(ds_name,streaming=True)

#ds_name="suolyer/pile_stackexchange"
#ds=load_dataset(ds_name,split="train",streaming=False).shuffle()

#ds_name="hatakeyama-llm-team/japanese2010"
#ds_name="hpprc/wikipedia-20240101"
#ds_name="sudy-super/JetCopper-10B"
# %%

In [75]:
n_layers=400
max_new_tokens=4000
model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"
bot=GGUFBot(model_path,max_new_tokens=max_new_tokens,n_ctx=max_new_tokens,n_gpu_layers=n_layers)

llama_model_loader: additional 3 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 29 key-value pairs and 563 tensors from /home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mixtral-8x22B-Inst...
llama_model_loader: - kv   2:                          llama.block_count u32              = 56
llama_model_loader: - kv   3:                       llama.context_length u32              = 65536
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 6144
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 16384
llama_model_loader: - 

loading model...


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   yes
ggml_cuda_init: CUDA_USE_TENSOR_CORES: no
ggml_cuda_init: found 8 CUDA devices:
  Device 0: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 2: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 3: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 4: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 5: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 6: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 7: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
llm_load_tensors: ggml ctx size =    2.51 MiB
llm_load_tensors: offloading 56 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 57/57 layers to GPU
llm_load_tensors:        CPU buffer size =   132.00 MiB
llm_load_tensors:      CUDA0 buffer size = 12238.73 MiB
llm_load_tenso

In [76]:

q_gen=SimpleQuestionGenerator()
a_gen=AnswerGenerator(bot)

In [86]:
record={}

record["text"]=next(ds)
record["inst_question"]=q_gen(record["text"])
record["question"]=bot.ask(record["inst_question"])
a_gen(record)
record

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1997.54 ms
llama_print_timings:      sample time =      21.53 ms /    90 runs   (    0.24 ms per token,  4180.02 tokens per second)
llama_print_timings: prompt eval time =    2092.21 ms /   239 tokens (    8.75 ms per token,   114.23 tokens per second)
llama_print_timings:        eval time =    4084.25 ms /    89 runs   (   45.89 ms per token,    21.79 tokens per second)
llama_print_timings:       total time =    6373.57 ms /   328 tokens
Llama.generate: prefix-match hit


In [ ]:


#bot=""

# %%
import json
import random
out_path="data/"+ds_name.split("/")[-1]+".jsonl"

#question=random.choice(original_question_list)



while True:
    question=next(loader)["text"]
    question=question[:random.randint(0,len(question))]
    question=question[random.randint(0,len(question)):]
    question=question[:500]

    q=question_template+question
    #try:
    if True:    
        new_question=bot.ask(q).replace("#質問","").replace("#問題","").replace("#指示","").strip()[:3000]

        d={"question":new_question,}
        ans_list=[]
        for i in range(2):
            inst=random.choice(inst_list)
            ans=bot.ask(inst+new_question)
            d[f"ans{i}"]=ans
        with open(out_path, 'a') as f:
            f.write(json.dumps(d,ensure_ascii=False)+"\n")
    #except:
    #    pass
    """
    question=new_question
    #question=question[:random.randint(0,len(question))]
    if random.randint(0,3)==0:
        question=question[:int(len(question)*0.8)]
    elif random.randint(0,3)==1:
        question=question[int(len(question)*0.2):]
    if random.randint(0,5)==0:
        question=random.choice(original_question_list)
    """
# %%

